In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
movie = pd.read_csv('../../datasets/pandas-cookbook/movie.csv', index_col='movie_title')
college = pd.read_csv('../../datasets/pandas-cookbook/college.csv')
employee = pd.read_csv('../../datasets/pandas-cookbook/employee.csv')

In [5]:
# percentage of movies with length greater than 120 minutes
# we drip NaNs because pandas evaluates them as False, 
# and we therefore get a slightly wrong result.
movie['duration'].dropna().gt(120).mean() * 100

21.199755152009793

### Constructing multiple Boolean conditions

In [4]:
criteria_a1 = movie.imdb_score > 8
criteria_a2 = movie.content_rating == 'PG-13'
criteria_a3 = (movie.title_year < 2000) | (movie.title_year > 2009)
criteria_a = criteria_a1 & criteria_a2 & criteria_a3

criteria_b1 = movie.imdb_score < 5
criteria_b2 = movie.content_rating == 'R'
criteria_b3 = (movie.title_year <= 2000) | (movie.title_year <= 2009)
criteria_b = criteria_b1 & criteria_b2 & criteria_b3

cols = ['imdb_score', 'content_rating', 'title_year']
movie.loc[criteria_a | criteria_b, cols].head()

,imdb_score,content_rating,title_year
movie_title,,,
The Dark Knight Rises,8.5,PG-13,2012.0
The Avengers,8.1,PG-13,2012.0
Captain America: Civil War,8.2,PG-13,2016.0
Guardians of the Galaxy,8.1,PG-13,2014.0
Interstellar,8.6,PG-13,2014.0


### Index filtering

In [5]:
college2 = college.set_index('STABBR').sort_index()
college2.loc['TX'].head(n=2)

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,
TX,Manuel and Theresa's School of Hair Design-Vic...,Victoria,0.0,0.0,0.0,0,NaN,NaN,0.0,30.0,...,0.0,0.0,0.0000,0.0000,1,0.6286,0.4571,0.5652,NaN,PrivacySuppressed
TX,Criswell College,Dallas,0.0,0.0,0.0,1,NaN,NaN,0.0,253.0,...,0.0,0.0,0.0672,0.4506,1,0.4468,0.2298,0.5064,NaN,PrivacySuppressed


### Translating SQL WHERE clauses

In [27]:
cols = ['UNIQUE_ID', 'DEPARTMENT', 'GENDER', 'BASE_SALARY']

top_five_depts = employee.DEPARTMENT.value_counts().index[:5]
criteria_dept = employee.DEPARTMENT.isin(top_five_depts)
criteria_gender = employee.GENDER == 'Female'
criteria_sal = employee.BASE_SALARY.between(80_000, 120_000)
criterias = criteria_dept & criteria_gender & criteria_sal

employee.loc[criterias, cols].head()

,UNIQUE_ID,DEPARTMENT,GENDER,BASE_SALARY
61,61,Houston Fire Department (HFD),Female,96668.0
66,66,Health & Human Services,Female,100791.0
136,136,Houston Police Department-HPD,Female,81239.0
265,265,Health & Human Services,Female,81849.0
304,304,Health & Human Services,Female,81972.0


### Improve the readability of Boolean indexing with the query method

Strings passed to the .query method are going to look more like plain English than normal pandas code. It is possible to reference Python variables using the at symbol (@), as with depts. All DataFrame column names are available in the query namespace by referencing their names without extra quotes. If a string is needed, such as Female, inner quotes will need to wrap it.

Another nice feature of the query syntax is the ability to combine Boolean operators using and, or, and not.



In [6]:
top_five_depts = employee.DEPARTMENT.value_counts().index[:5]
cols = ['UNIQUE_ID', 'DEPARTMENT', 'GENDER', 'BASE_SALARY']

qs = (
    'DEPARTMENT in @top_five_depts '
    ' and GENDER == "Female"'
    ' and 80000 <= BASE_SALARY <= 120000' 
)

emp_filtered = employee.query(qs)
emp_filtered[cols].head()

,UNIQUE_ID,DEPARTMENT,GENDER,BASE_SALARY
61,61,Houston Fire Department (HFD),Female,96668.0
66,66,Health & Human Services,Female,100791.0
136,136,Houston Police Department-HPD,Female,81239.0
265,265,Health & Human Services,Female,81849.0
304,304,Health & Human Services,Female,81972.0


### Preserving Series size with the .where method

In [23]:
fb_likes = movie.actor_1_facebook_likes.dropna()
criteria_high = fb_likes < 20_000
criteria_low = fb_likes > 300
fb_likes.where(criteria_high, 20_000).where(criteria_low, 300).head()

movie_title
Avatar                                         1000.0
Pirates of the Caribbean: At World's End      20000.0
Spectre                                       11000.0
The Dark Knight Rises                         20000.0
Star Wars: Episode VII - The Force Awakens      300.0
Name: actor_1_facebook_likes, dtype: float64

In [24]:
# we could have done the same using `clip`
fb_likes.clip(lower=300, upper=20_000).head()

movie_title
Avatar                                         1000.0
Pirates of the Caribbean: At World's End      20000.0
Spectre                                       11000.0
The Dark Knight Rises                         20000.0
Star Wars: Episode VII - The Force Awakens      300.0
Name: actor_1_facebook_likes, dtype: float64

### Mask

In [11]:
cols = ['movie_title', 'title_year']
c1 = movie.title_year >= 2010
c2 = movie.title_year.isna()
criteria = c1 | c2

# By default the .mask method fills in rows where the Boolean array is True with NaN.
# mask sets title_year of all movies created in 2010 and after to NaN
# we use .dropna to remove these rows
movie[['title_year']].mask(criteria).dropna(how='all').head()

,title_year
movie_title,
Avatar,2009.0
Pirates of the Caribbean: At World's End,2007.0
Spider-Man 3,2007.0
Harry Potter and the Half-Blood Prince,2009.0
Superman Returns,2006.0
